In [30]:
%matplotlib inline
import matplotlib.pyplot as plt
import pandas
from scipy.optimize import curve_fit 
from numpy import log, exp, linspace, sqrt, diag
from sklearn.linear_model import LinearRegression
import os
import re 

df = pandas.read_csv( 'data/assay_data_clean.csv' )
df.index = df.mutant
df.drop( 'mutant', inplace=True )
df.drop( ['WT'], inplace=True )
print len( df ) 

def is_good( name ):
    return os.path.isfile( '/Users/alex/Documents/bagel-orders/oligos/{}.fasta'.format( name.lower() ) )

l = []
for name in df.index:
    if is_good( name ):
        l.append( name )
    else:
        new = name[0] + str( int( re.findall( r'\d+', name )[0] ) - 3 ) + name[-1]
        if is_good( new ):
            l.append( new )
        else:
            l.append( 'drop_me' )
            
df.index = l
df.drop( ['drop_me'], inplace=True )
print len( df ) 

# temperature keys as CSV files in data/
temp_key = pandas.read_csv( 'data/temp_key.csv' )
temp_key.index = temp_key.Well
df['temp'] = df.well.str[0].map( temp_key.Celsius.to_dict() ) 
df.dropna( inplace=True )

2209
2089


# Curve fit to logistic equation

In [21]:
# logistic equation 
def f(x, x0, k): 
    return 1/(1+exp(-k*(x-x0)))

# util function to fit a mutant 
def fit( df ):
    name = df.mutant.unique()[0]
    df.rate = df.rate / df.rate.max()
    
    # linear fit 
    reg = LinearRegression()
    reg.fit( df.temp.reshape(-1, 1), df.rate )
    slope = reg.coef_[0]
    
    # try fitting to logistic eqn 
    try:
        p0 = ( df.temp.mean(), slope )
        popt, pcov = curve_fit( f, df.temp, df.rate, p0=p0 )
        perr = sqrt( diag( pcov ) )
        if 30 < popt[0] < 50: # biological assay limits 
            return pandas.Series( popt, index=['tm', 'k'] )
        else:
            #iffy += [ name ]
            raise Exception( 'Yo yo yo, {} is outside range'.format( popt[0] ) )
    except Exception as e:
        print e
    
grouped = df.groupby( by='mutant' )
fits = grouped.apply( fit )
print len( fits )

Yo yo yo, -185.358619149 is outside range
Yo yo yo, 10.4195936133 is outside range
Yo yo yo, 27.2125573509 is outside range
Yo yo yo, 20.7324542558 is outside range
Yo yo yo, -450.877938708 is outside range
Yo yo yo, 326.284708587 is outside range
Yo yo yo, 85.9030545081 is outside range
Yo yo yo, 5.89781401266 is outside range
Yo yo yo, 139.600056463 is outside range
Yo yo yo, -3.46074953682 is outside range
Yo yo yo, 17.1018681957 is outside range
Yo yo yo, 85.9030545081 is outside range
Yo yo yo, 52.3986951672 is outside range
57


In [22]:
fits

,tm,k
mutant,,
A195S,39.082598,-0.906300
A356A,39.888266,-1.124698
A357A,39.094581,-0.325209
A408A,39.955931,-1.260728
C167Q,38.471618,-0.532181
D403A,NaN,NaN
E154D,38.698871,-0.699229
E164A,NaN,NaN
E177A,37.307311,-0.487548


## Diagnostic plots to help determine goodness of fits

In [23]:
for index, df in grouped:
    name = df.mutant.unique()[0]
    rate = df.rate / df.rate.max() # have to do this again
    plt.figure( figsize=(2,2) )
    plt.scatter( df.temp, rate, alpha=0.7, color='black', marker='.' )
    popt = fits.loc[ name ]
    if popt is not None:
        x_space = linspace( df.temp.min(), df.temp.max(), 100 )
        plt.plot( x_space, f( x_space, *popt ), alpha=0.5, color='purple' )
    plt.xlabel( 'T (C)' )
    plt.ylabel( 'Normalized rate')
    plt.xticks( [ 30, 40, 50 ] )
    plt.yticks( [ 0, 0.5, 1 ] )
    plt.title( name )
    plt.tight_layout()
    plt.savefig( 'plots/%s.pdf' % name, format='pdf' )
    #plt.show()
    plt.close()

## Clean up and apply what we learned from the plots to the data set 

In [24]:
# nothing here, yet! 

In [27]:
# add metadata
fits['native'] = fits.index.str[0]
fits['designed'] = fits.index.str[-1]
fits['position'] = fits.index.str[1:-1].astype( int )

# print some summary stats
print len( fits )
print fits

57
               tm         k native designed  position
mutant                                               
A195S   39.082598 -0.906300      A        S       195
A356A   39.888266 -1.124698      A        A       356
A357A   39.094581 -0.325209      A        A       357
A408A   39.955931 -1.260728      A        A       408
C167Q   38.471618 -0.532181      C        Q       167
D403A         NaN       NaN      D        A       403
E154D   38.698871 -0.699229      E        D       154
E164A         NaN       NaN      E        A       164
E177A   37.307311 -0.487548      E        A       177
E180K   36.646840 -0.671301      E        K       180
E180L   39.223503 -0.520210      E        L       180
E406D   40.545785 -1.016127      E        D       406
E426S   39.459240 -1.449480      E        S       426
F75H          NaN       NaN      F        H        75
H101R   40.021708 -0.910848      H        R       101
H122E         NaN       NaN      H        E       122
H122N         NaN       N

In [28]:
# makes resfiles for DDG Rosetta app 
for ( index, series ) in fits.iterrows():
    with open( 'ddg/resfiles/{}'.format( index ), 'w' ) as fn:
        fn.write( 'NATRO\nSTART\n{} A PIKAA {}'.format( series.position, series.designed ) )

In [29]:
!ls

bagel-thermostability.ipynb data                        ddg                         plots
